<a href="https://colab.research.google.com/github/Maureentonui/Politeness-ML-project/blob/main/Another_copy_of_notebook2c5809837c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.pipeline import Pipeline

df =pd.read_csv("/content/sample_data/politeness_train.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4914 entries, 0 to 4913
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   utterance_id     4914 non-null   int64 
 1   conversation_id  4914 non-null   int64 
 2   text             4914 non-null   object
 3   polite           4914 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 153.7+ KB


,utterance_id,conversation_id,text,polite
0,3208,2099,Is it for a particular CPU (e.g. a Z80 or the ...,1
1,4458,4622,What is an eastern style pot? Is this a cup-l...,1
2,4888,5599,I didn't know that you had to use an email add...,0
3,3316,2295,I don't understand. That call stack is all ab...,1
4,4005,3674,The last sentence makes me think that there is...,0


In [16]:
drop_cols=['utterance_id','conversation_id']
df = df.drop(drop_cols, axis = 1)
df.head()

,text,polite
0,Is it for a particular CPU (e.g. a Z80 or the ...,1
1,What is an eastern style pot? Is this a cup-l...,1
2,I didn't know that you had to use an email add...,0
3,I don't understand. That call stack is all ab...,1
4,The last sentence makes me think that there is...,0


In [17]:
df.polite.value_counts()

1    2457
0    2457
Name: polite, dtype: int64

In [18]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


def preprocess_text(text):
    # Remove non-alphanumeric characters and URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'[^\w\s,]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

text = "This is likely to be a shell misconfiguration, probably the lack of prompt protection [suggested by mikel](http://unix.stackexchange.com/questions/6580/why-is-gnu-screen-byobu-leaving-garbage-text-in-the-shell-during-a-reverse-sear/6581#6581). What shell are you using (probably bash, but please confirm)?"
clean_text = preprocess_text(text)
print("Cleaned Text:", clean_text)

Cleaned Text: likely shell misconfiguration probably lack prompt protection suggested mikel shell using probably bash please confirm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
df.text = df.text.map(preprocess_text)
df.head()


,text,polite
0,particular cpu eg z cell processor spu abstrac...,1
1,eastern style pot cuplike device lid strain le...,1
2,didnt know use email address instead username ...,0
3,dont understand call stack debug help library ...,1
4,last sentence make think bigger picture really...,0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string


stops = nltk.corpus.stopwords.words('english')

vectorizer = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize, stop_words=stops)
vectorizer.fit(train['text']) # corpus is a list of strings (documents)
bow_train = vectorizer.transform(train['headline'])
bow_dev = vectorizer.transform(dev['headline'])
bow_test = vectorizer.transform(test['headline'])

# New Section